In [8]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#beautifulsoup
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

import re

import json

import pandas as pd

In [9]:
driver = webdriver.Chrome(
)
driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx")

#等待頁面載入
try:
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, 'input#jud_jmain')
    ))
    input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
    input_case_reason.send_keys("殺人")
    input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
    input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")
    search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
    search_button.click()
    sleep(1)
except TimeoutException:
    print("can not search")

In [10]:
#等待iframe與下一頁按鈕
try:
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, "iframe#iframe-data")
    ))

    iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
    driver.switch_to.frame(iframe)
    page_url = driver.find_element(By.CSS_SELECTOR, 'a#hlNext').get_attribute('href')
    print(page_url)

except TimeoutException:
    print("not found...next_page_button")

https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in


#測試一下標籤
driver.get(page_url)
cssSelector = 'a#hlTitle'
case_url = driver.find_element(By.CSS_SELECTOR, cssSelector)
case_url.click()
# 如果這邊click之後，沒有等JSt生成，就直接page_source抓動態html的話，會抓不到，用標籤判斷根本沒用，標籤會在，只是沒有內容，看有沒有其他隱性等待的方法(這樣比較保險)，不然就要強制ˋ等待了，或使隱性等待?(但他是如何判斷加載完成的...)
#!!!!要抓li，js生成的是li，因為平常進網頁就載入好了，跟一班明顯的js變化不同，所以沒有察覺
try:
    # 看有沒有抓到js生成的側攔
    sidebarSelector = 'ul.rela-law li'
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, sidebarSelector)
    ))

    #有的話，就取得動態html
    case_html = driver.page_source
    soup = bs(case_html, "lxml")

except TimeoutException:
    print("not found...element")


# side_bar = soup.select_one('div.rela-area.col-xs-4')
# print(side_bar.get_text())
### 這邊要改li然後再加進一個Lsit當中
linesSelector = 'ul.rela-law li'
lines = soup.select(linesSelector)
for law in lines:
    print(law.get_text())



# 若case_id為"本件無歷審裁判"，則不會有url
history_judgement = soup.select("div#JudHis div.panel-body ul li")
history_judgement_list = []
for index in range(0, len(history_judgement)):
    if not str(history_judgement[index].select_one('a')) == "None":
        history_judgement_list.append({
        "case_id":str(history_judgement[index].get_text()),
        "url":"https://judgment.judicial.gov.tw/FJUD/"+ str(history_judgement[index].select_one('a')['href'])
        #抓不到a就會變成NoneType object，要怎麼處理??
        # NoneType object當然也不能用['href']所以會報錯

        # "url":"https://judgment.judicial.gov.tw/FJUD/"+ history_judgement[index].select_one('a')['href']
        })
    else:
        history_judgement_list.append({
        "case_id":str(history_judgement[index].get_text())
        })

history_judgement_list

In [11]:
page_url_list = [] 
regex_page_url = "&page=[\d]*"
test = re.split(regex_page_url, page_url)
for each_page in range(1,26):
    page_url_list.append(f"{test[0]}&page={each_page}{test[1]}") 

In [12]:
# 放所有case的資料
all_case_list = []

In [13]:
# 加try excecpt
# 要設一些如果沒抓到東西時的條件，比如歷史審判可能會沒有
# 然後改移下一開始的wait反正就是等旁偏那個javascript生出來的測攔出來就好
# 開始爬case，這是每一頁的網址
for page in page_url_list:
    driver.get(page)
    try:
        cssSelector = 'a#hlTitle'
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, cssSelector)
        ))

        case_url = driver.find_elements(By.CSS_SELECTOR, cssSelector)
    except TimeoutException:
        print("cases not found")
        break

    # cases找到後開始一個一個點
    # 這是每一頁當中的至多20筆的資料    
    for case in case_url:
        case.click()

        try:
            # 看有沒有抓到js生成的側攔"當中的li"，li就是js後來生成的
            # 這邊要改可能還是沒有laws，而且結果超奇怪....根本抓不到law結果欄位有值，抓地到history但沒有值...
            
            side_li_selector_2 = 'div#JudHis div.panel-body ul li'
            WebDriverWait(driver, 15).until(
            EC.presence_of_element_located(
            (By.CSS_SELECTOR, side_li_selector_2)
            ))

            #還是這邊可以等兩個?等個兩個載好在來取得動態html
        except TimeoutException:
            # 如果有錯的話抓這個webelement的text(標題)，href(超連結)
            print(f"{case.text} {case.get_attribute('href')} not found...history, may be critical error")
            #這邊之後要改logging

        try:   
            #不然這邊就是只看歷史審判(因為1.有時候很多所以加載比較慢2.好像一定會有此標籤因為至少也是"本案例無相關歷史案件"之類的)
            # 想一下這邊要用主動等待還是強制，主動等待的好處在於有可能是網路加載比較慢(可以避免這個問題)，而且真的沒這個欄位的時間應該很少
            # 但這個欄位真的有可能抓不到那就要抱錯，可是我們應該分開來報，因為兩個的問題不一樣
            side_li_selector_1 = 'ul.rela-law li'
            WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
            (By.CSS_SELECTOR, side_li_selector_1)
            ))
        except TimeoutException:
            print(f"{case.text} {case.get_attribute('href')} not found...laws, it's ok")


        #取得動態html #反正都要連到這個case當中，若上面兩個沒抓到，就只是side_bar的資料有少但其他資料還是有的
        #但我就在想這樣會不會很雞肋、而且反而拖到時間，因為有些case是真的沒有上面那兩個side_bar，還是乾脆就強制等待sleep算了，不知道哪種比較優且快
        sleep(1) # 稍微等一下laws那個side_bar位置
        case_html = driver.page_source
        soup = bs(case_html, "lxml")

        allSelector = 'div.col-td'
        all = soup.select(allSelector)

        # 這邊要加個判斷，不知道寫的可不可以，不是抓不到但好像怪怪的
        laws_list = []
        linesSelector = 'ul.rela-law li'
        lines = soup.select(linesSelector)
        for law in lines:
            laws_list.append(str(law.get_text()))

        
        history_judgement = soup.select("div#JudHis div.panel-body ul li")
        history_judgement_list = []
        for index in range(0, len(history_judgement)):
            if not str(history_judgement[index].select_one('a')) == "None":
                history_judgement_list.append({
                    "case_id":str(history_judgement[index].get_text()),
                    "url":"https://judgment.judicial.gov.tw/FJUD/"+ str(history_judgement[index].select_one('a')['href'])
                })
            else:
                history_judgement_list.append({
                "case_id":str(history_judgement[index].get_text())
                })


        all_case_list.append({
        "case_id":str(all[0].get_text().strip()),
        "date":str(all[1].get_text().strip()),
        "simple_reason":str(all[2].get_text().strip()),
        "laws":laws_list,
        "history_judgement":history_judgement_list,
        "context":str(all[3].get_text()),
        })

        driver.back()


In [14]:
print(all_case_list[44]['laws'])

['家庭暴力防治法 第 2、3 條（104.02.04）', '刑事訴訟法 第 156、158.2、158.4、159.1、159.2、159.5、299 條（106.11.16）', '中華民國刑法 第 1、25、38.1、95、271、320 條（105.11.30）']


In [15]:
print(len(all_case_list))

500


In [16]:
# 就寫成dataframe再轉成csv八
tt = pd.DataFrame(all_case_list)
tt.to_csv('./cases_500_v4.csv')

In [17]:
# side_bar兩個都要判斷?
# div#JudHis > div.panel-body > ul > li

In [18]:
tt.head(3)

,case_id,date,simple_reason,laws,history_judgement,context
0,臺灣新北地方法院 113 年度訴字第 179 號刑事判決,民國 113 年 04 月 25 日,殺人未遂,"[刑事訴訟法 第 1、158.4、159.5、163.2、299 條（106.11.16）,...",[{'case_id': '臺灣新北地方法院 113 年度 訴 字第 179 號判決(113...,\n\n\n\n臺灣新北地方法院刑事判決113年度訴字第179號公 訴 人 臺灣新北地...
1,臺灣高等法院 112 年度上訴字第 4225 號刑事判決,民國 113 年 04 月 24 日,殺人未遂等,"[槍砲彈藥刀械管制條例 第 7、12 條（106.06.14）, 中華民國刑法 第 1、25...",[{'case_id': '臺灣新竹地方法院 112 年度 訴 字第 75 號裁定(112....,\n\n\n\n臺灣高等法院刑事判決112年度上訴字第4225號上 訴 人 臺灣新竹地...
2,臺灣高等法院 臺中分院 113 年度上訴字第 213 號刑事判決,民國 113 年 04 月 24 日,殺人未遂等,"[刑事訴訟法 第 159.1、369 條（106.11.16）, 中華民國刑法 第 1、13...",[{'case_id': '臺灣臺中地方法院 112 年度 訴 字第 1877 號裁定(11...,\n\n\n\n 臺灣高等法院臺中分院刑事判決113年度上訴字第213號上 訴 人 臺...
